<a href="https://colab.research.google.com/github/monika123-S/Dimension-Reduction-Using-K-Means-for-Activity-Recognition/blob/main/Dimensionality_reduction_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.

# Display first few rows of the dataset

print(df.head())

# Summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Data type information
print(df.info())

<ipython-input-3-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-3-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


        0         1         2         3         4         5         6    \
0  0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
1  0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
2  0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
3  0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
4  0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   

        7         8         9    ...       551       552       553       554  \
0 -0.983185 -0.923527 -0.934724  ... -0.074323 -0.298676 -0.710304 -0.112754   
1 -0.974914 -0.957686 -0.943068  ...  0.158075 -0.595051 -0.861499  0.053477   
2 -0.963668 -0.977469 -0.938692  ...  0.414503 -0.390748 -0.760104 -0.118559   
3 -0.982750 -0.989302 -0.938692  ...  0.404573 -0.117290 -0.482845 -0.036788   
4 -0.979672 -0.990441 -0.942469  ...  0.087753 -0.351471 -0.699205  0.123320   

        555       556       557       558       559       560  
0  0

In [ ]:
# Task 2: Encode class labels
# YOUR CODE HERE: Use LabelEncoder to encode class labels

label_encoder = LabelEncoder()
encoded_y =label_encoder.fit_transform(y.values.ravel())

In [ ]:
# Task 3: Scale the features using StandardScaler
# YOUR CODE HERE: Apply StandardScaler to df

scaler = StandardScaler()
df_scaled =scaler.fit_transform(df) # YOUR CODE HERE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB
import time

start_time=time.time()
# 1. Create a pipeline using Gaussian Naive Bayes
pipeline = Pipeline([('classifier', GaussianNB())])
pipeline.fit(X_train, y_train.values.ravel())
y_pred = pipeline.predict(X_test)
end_time=time.time()
print(f"Time taken: {end_time - start_time} seconds")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Time taken: 0.05854058265686035 seconds
Accuracy: 0.7314751869476547


In [16]:
'''# TASK 7 - K-Means for dimensionality reduction
n_clusters = 50  # Choose an appropriate number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = []
for i in range(n_clusters):
    cluster_indices = np.where(kmeans.labels_ == i)[0]
    selected_features_indices.append(cluster_indices[np.argmax(np.var(df_scaled[:, cluster_indices], axis=0))])
selected_features = df.iloc[:, selected_features_indices]
print(selected_features.head())'''

        210       10        37        88        23        453       298  62   \
0 -0.022897 -0.567378  0.376314 -0.992818 -0.679338  0.547151 -0.220547 -1.0   
1 -0.109366 -0.557851 -0.013429 -0.991277 -0.500930 -0.068178 -0.049709 -1.0   
2 -0.074212 -0.557851 -0.124698 -0.987238 -0.485821 -0.110720  0.163063 -1.0   
3  0.046396 -0.576159 -0.305693 -0.991483 -0.850930 -0.049728 -0.460915 -1.0   
4 -0.277657 -0.569174 -0.155804 -0.990686 -0.559477 -0.162230 -0.180383 -1.0   

        44        9    ...       337       78        161       379       156  \
0 -0.981708 -0.934724  ... -0.995146  0.439027 -0.055517 -0.507345 -0.331831   
1 -0.989447 -0.943068  ... -0.998754 -0.865711 -0.044819 -0.655535 -0.149429   
2 -0.992866 -0.938692  ... -0.999912  0.337936 -0.042410 -0.803407  0.184349   
3 -0.981393 -0.938692  ... -0.999984 -0.968821 -0.036333 -0.577038  0.202092   
4 -0.988098 -0.942469  ... -0.999436 -0.184840 -0.037633 -0.764369  0.473251   

        75        81        30        

In [15]:
# SOLUTION -  K-Means for dimensionality reduction
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df_scaled[:, selected_features_indices]
print(selected_features)

[[ 2.57602421 -0.94666714 -0.7826638  ... -0.71840922  2.38058271
   0.05426879]
 [ 1.100926   -1.14021606 -0.78177854 ...  0.24525519  1.24214562
  -0.28975396]
 [ 0.51449745 -1.2108155  -0.78026501 ... -0.67367648  1.66521941
   0.19924571]
 ...
 [-1.87078992  1.06645255  1.02663371 ...  0.4560065   0.12600117
  -0.13816968]
 [-2.53817908  0.81598078  0.9767199  ...  0.23656391  0.4244078
  -1.17094764]
 [-1.82848539  0.84970303  1.15550879 ...  0.15363336 -0.43604787
  -1.00669305]]


In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

In [18]:
X_train_selected, X_test_selected, y_train, y_test = train_test_split(
    selected_features, encoded_y, test_size=0.2, random_state=42
)

In [21]:
start_time = time.time()
gnb_classifier = GaussianNB()
gnb_classifier.fit(X_train_selected, y_train)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

Time taken: 0.013953924179077148 seconds


In [20]:
y_pred_gnb = gnb_classifier.predict(X_test_selected)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
print(f"Accuracy of Gaussian Naive Bayes: {accuracy_gnb}")

Accuracy of Gaussian Naive Bayes: 0.8218898708361658
